In [47]:
import numpy as np
import pandas as pd
from arch import arch_model
from metrics import eval_preds, composite_score, quantiles


df = pd.read_csv("spy_data.csv", index_col=0, parse_dates=[0])
df = df[["log_ret", "rv5", "bv"]].copy()

# Scale returns to percent 
df["r"] = 100.0 * df["log_ret"]

In [48]:
import numpy as np
import pandas as pd
from metrics import eval_preds, composite_score, quantiles
import matplotlib.pyplot as plt


def build_har_frame(df, w=5, m=22, include_jump=False, eps=1e-12):

    out = df.copy()
    out["log_rv"] = np.log(out["rv5"] + eps)

    X = pd.DataFrame(index=out.index)
    X["log_rv_d"] = out["log_rv"]
    X["log_rv_w"] = out["log_rv"].rolling(w).mean()
    X["log_rv_m"] = out["log_rv"].rolling(m).mean()

    if include_jump:
        out["jump"] = np.maximum(out["rv5"] - out["bv"], 0.0)

    y = out["log_rv"].shift(-1).rename("log_rv_next")       
    r_next = out["log_ret"].shift(-1).rename("r_next") 

    har = pd.concat([X, y, r_next], axis=1).dropna()
    return har



### Usefull functions

In [49]:
from scipy import stats

def fit_return_quantiles(r_train_next, sigma_train_next, method="normal", alphas=(0.01,0.05,0.10)):

    if method == "normal":
        return {a: stats.norm.ppf(a) for a in alphas}

    z = (r_train_next / sigma_train_next)
    z = z[np.isfinite(z)]

    if method == "fhs":
        return {a: np.quantile(z, a) for a in alphas}

    if method == "t":

        nu, loc, scale = stats.t.fit(z)
        return {a: stats.t.ppf(a, df=nu, loc=loc, scale=scale) for a in alphas}

    raise ValueError("method must be 'normal', 't', or 'fhs'")


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge

def cv_score_one_config(df, w, m, include_jump, reg, reg_alpha, q_method,
                       alphas=(0.01,0.05,0.10), n_splits=5):
    har = build_har_frame(df, w=w, m=m, include_jump=include_jump)

    X_cols = [c for c in har.columns if c not in ["log_rv_next", "r_next"]]
    X = har[X_cols].values
    y_logrv = har["log_rv_next"].values
    r_next = har["r_next"].values

    tscv = TimeSeriesSplit(n_splits=n_splits)

    fold_metrics_store = []
    for train_idx, val_idx in tscv.split(X):
        X_tr, X_va = X[train_idx], X[val_idx]
        y_tr, y_va = y_logrv[train_idx], y_logrv[val_idx]
        r_tr, r_va = r_next[train_idx], r_next[val_idx]

        if reg == "ols":
            model = LinearRegression()
        elif reg == "ridge":
            model = Ridge(alpha=reg_alpha)
        else:
            raise ValueError("reg must be 'ols' or 'ridge'")

        model.fit(X_tr, y_tr)

        pred_tr_rv = np.exp(model.predict(X_tr))
        pred_va_rv = np.exp(model.predict(X_va))
        sigma_tr = np.sqrt(pred_tr_rv)
        sigma_va = np.sqrt(pred_va_rv)

        q_alpha = fit_return_quantiles(r_tr, sigma_tr, method=q_method, alphas=alphas)

        var_pred = {a: sigma_va * q_alpha[a] for a in alphas}

        metrics_val = eval_preds(r_va, var_pred, alphas)
        fold_metrics_store.append(metrics_val)

    avg_metrics = {}
    for a in alphas:
        avg_metrics[a] = {
            "actual_coverage": float(np.mean([fm[a]["actual_coverage"] for fm in fold_metrics_store])),
            "pinball_loss": float(np.mean([fm[a]["pinball_loss"] for fm in fold_metrics_store])),
            "kupiec_pval": float(np.nanmean([fm[a]["kupiec_pval"] for fm in fold_metrics_store])),
        }

    avg_score = float(np.mean([composite_score(fm, alphas) for fm in fold_metrics_store]))
    
    return avg_score, avg_metrics


### Tuning

In [ ]:
grid = []
for w, m in [(5,22), (10,22), (5,63), (10,63)]:    
    for include_jump in [False, True]:
        for reg, reg_alpha in [("ridge", 1e-3), ("ridge", 1e-2), ("ridge", 1e-1)]:
            for q_method in ["normal", "t"]:
                grid.append((w, m, include_jump, reg, reg_alpha, q_method))



In [52]:
rows = []
alphas_grid = (0.01, 0.05, 0.10)

for (w, m, include_jump, reg, reg_alpha, q_method) in grid:
    avg_score, avg_metrics = cv_score_one_config(
        df, w=w, m=m, include_jump=include_jump,
        reg=reg, reg_alpha=reg_alpha,
        q_method=q_method,
        alphas=alphas_grid,
        n_splits=5
    )
    
    rows.append({
        "w": w, "m": m, "include_jump": include_jump,
        "reg": reg, "reg_alpha": reg_alpha, "q_method": q_method,
        "score": avg_score,
        "cov_1%": avg_metrics[0.01]["actual_coverage"],
        "cov_5%": avg_metrics[0.05]["actual_coverage"],
        "cov_10%": avg_metrics[0.10]["actual_coverage"],
        "pin_1%": avg_metrics[0.01]["pinball_loss"],
        "pin_5%": avg_metrics[0.05]["pinball_loss"],
        "pin_10%": avg_metrics[0.10]["pinball_loss"],
        "pval_1%": avg_metrics[0.01]["kupiec_pval"],
        "pval_5%": avg_metrics[0.05]["kupiec_pval"],
        "pval_10%": avg_metrics[0.10]["kupiec_pval"],
    })
    print(f"Done w={w}, m={m}, jump={include_jump}, reg={reg}, alpha={reg_alpha:.4f}, q_method={q_method} -> score={avg_score:.4f}")

res_grid = pd.DataFrame(rows).sort_values("score").reset_index(drop=True)
res_grid.head(10)

Done w=5, m=22, jump=False, reg=ridge, alpha=0.0010, q_method=normal -> score=0.0044
Done w=5, m=22, jump=False, reg=ridge, alpha=0.0010, q_method=t -> score=0.0024
Done w=5, m=22, jump=False, reg=ridge, alpha=0.0100, q_method=normal -> score=0.0044
Done w=5, m=22, jump=False, reg=ridge, alpha=0.0100, q_method=t -> score=0.0024
Done w=5, m=22, jump=False, reg=ridge, alpha=0.1000, q_method=normal -> score=0.0044
Done w=5, m=22, jump=False, reg=ridge, alpha=0.1000, q_method=t -> score=0.0024
Done w=5, m=22, jump=True, reg=ridge, alpha=0.0010, q_method=normal -> score=0.0044
Done w=5, m=22, jump=True, reg=ridge, alpha=0.0010, q_method=t -> score=0.0024
Done w=5, m=22, jump=True, reg=ridge, alpha=0.0100, q_method=normal -> score=0.0044
Done w=5, m=22, jump=True, reg=ridge, alpha=0.0100, q_method=t -> score=0.0024
Done w=5, m=22, jump=True, reg=ridge, alpha=0.1000, q_method=normal -> score=0.0044
Done w=5, m=22, jump=True, reg=ridge, alpha=0.1000, q_method=t -> score=0.0024
Done w=10, m=22,

,w,m,include_jump,reg,reg_alpha,q_method,score,cov_1%,cov_5%,cov_10%,pin_1%,pin_5%,pin_10%,pval_1%,pval_5%,pval_10%
0,10,63,True,ridge,0.001,t,0.002250,0.014698,0.055381,0.098425,0.000316,0.001123,0.001840,0.052821,0.184773,0.478407
1,10,63,False,ridge,0.001,t,0.002250,0.014698,0.055381,0.098425,0.000316,0.001123,0.001840,0.052821,0.184773,0.478407
2,10,63,True,ridge,0.010,t,0.002250,0.014698,0.055381,0.098425,0.000316,0.001123,0.001840,0.052821,0.184773,0.478407
3,10,63,False,ridge,0.010,t,0.002250,0.014698,0.055381,0.098425,0.000316,0.001123,0.001840,0.052821,0.184773,0.478407
4,10,63,True,ridge,0.100,t,0.002250,0.014698,0.055381,0.098425,0.000316,0.001123,0.001840,0.052821,0.184773,0.478407
5,10,63,False,ridge,0.100,t,0.002250,0.014698,0.055381,0.098425,0.000316,0.001123,0.001840,0.052821,0.184773,0.478407
6,10,22,True,ridge,0.001,t,0.002298,0.014564,0.054876,0.098570,0.000318,0.001126,0.001844,0.055936,0.190173,0.417044
7,10,22,False,ridge,0.001,t,0.002298,0.014564,0.054876,0.098570,0.000318,0.001126,0.001844,0.055936,0.190173,0.417044
8,10,22,True,ridge,0.010,t,0.002298,0.014564,0.054876,0.098570,0.000318,0.001126,0.001844,0.055936,0.190173,0.417044
9,10,22,False,ridge,0.010,t,0.002298,0.014564,0.054876,0.098570,0.000318,0.001126,0.001844,0.055936,0.190173,0.417044


In [53]:
from scipy.stats import chi2

# Extract best hyperparams from res_grid
best_row = res_grid.iloc[0]
w_best = int(best_row["w"])
m_best = int(best_row["m"])
include_jump_best = best_row["include_jump"]
reg_best = best_row["reg"]
reg_alpha_best = best_row["reg_alpha"]
q_method_best = best_row["q_method"]

print("Best config:")
print(f"  w={w_best}, m={m_best}, include_jump={include_jump_best}")
print(f"  reg={reg_best}, reg_alpha={reg_alpha_best:.4f}, q_method={q_method_best}")
print(f"  CV score={best_row['score']:.4f}")

#Build HAR dataset with BEST hyperparams

har = build_har_frame(df, w=w_best, m=m_best, include_jump=include_jump_best)
X_cols = [c for c in har.columns if c not in ["log_rv_next", "r_next"]]

Best config:
  w=10, m=63, include_jump=True
  reg=ridge, reg_alpha=0.0010, q_method=t
  CV score=0.0022


### Fit final HAR regression on Train+Val

In [ ]:
n = len(har)
i_trainval = int(0.85 * n)

trainval = har.iloc[:i_trainval].copy()
test     = har.iloc[i_trainval:].copy()

X_tr = trainval[X_cols].values
y_tr = trainval["log_rv_next"].values
r_tr = trainval["r_next"].values

X_te = test[X_cols].values
r_te = test["r_next"].values

In [ ]:
if reg_best == "ols":
    model = LinearRegression()
elif reg_best == "ridge":
    model = Ridge(alpha=reg_alpha_best)


model.fit(X_tr, y_tr)

In [ ]:
sigma_tr = np.sqrt(np.exp(model.predict(X_tr)))
sigma_te = np.sqrt(np.exp(model.predict(X_te)))

In [ ]:
alphas = (0.01, 0.05, 0.10)
q_alpha = fit_return_quantiles(r_tr, sigma_tr, method=q_method_best, alphas=alphas)

var_test = pd.DataFrame(index=test.index)
var_test["r_next"] = r_te
for a in alphas:
    var_test[f"VaR_{int(100*a)}"] = sigma_te * q_alpha[a] 
 

### TEST backtest

In [ ]:
preds = {a: var_test[f"VaR_{int(100*a)}"].values for a in alphas}
metrics_by_q = eval_preds(var_test["r_next"].values, preds, alphas)
comp = composite_score(metrics_by_q, alphas)

results_summary = []
for a in alphas:
    m = metrics_by_q[a]
    pval = m["kupiec_pval"]
    results_summary.append({
        "Alpha": f"{int(100*a)}%",
        "Expected_Coverage": f"{100*a:.1f}%",
        "Actual_Coverage": f"{100*m['actual_coverage']:.2f}%",
        "Pinball_Loss": f"{m['pinball_loss']:.6f}",
        "Kupiec_LR": f"{m['kupiec_LR']:.2f}",
        "Kupiec_p_value": f"{pval:.4f}",
        "Test_Result": "PASS" if (np.isfinite(pval) and pval > 0.05) else "FAIL",
    })

results_df = pd.DataFrame(results_summary)
print(results_df.to_string(index=False))
print(f"\nComposite score: {comp:.4f}")


TEST backtest:
Alpha Expected_Coverage Actual_Coverage Pinball_Loss Kupiec_LR Kupiec_p_value Test_Result
   1%              1.0%           1.60%     0.000306      2.12         0.1453        PASS
   5%              5.0%           5.82%     0.000856      0.93         0.3345        PASS
  10%             10.0%          10.19%     0.001351      0.03         0.8690        PASS

Composite score: 0.0014
